In [3]:
import h5py, os, numpy as np

In [4]:
def add_tensor_component_to_spectre_group(hdf_group, tensors, order_on='connectivity'):
    """This function takes in a pointer to an open HDF5 file group, within
    which data is to be stored. It also takes in a dictionary of tensor
    components. The argument `order_on` ensures that the components are 
    populated in the right order contiguously.
    """
    if order_on is not None:
        assert order_on in hdf_group, "{0} not present in the HDF group provided".format(order_on)
        assert order_on in tensors, "{0} not present in the tensors provided".format(order_on)
        o1 = hdf_group[order_on][()]
        o2 = tensors[order_on]
        
    for t in tensors:
        if t in hdf_group:
            continue
        hdf_group.create_dataset(t, data=tensors[t])
    return hdf_group

def combine_element_wise_spectre_data(hdf_group,
                                      include='all', exclude=[],
                                      combined_tensors={},
                                      verbose=True):
    """This function takes in a pointer to an open HDF5 file group, within
    which is data stored in spectre's ExtentsAndTensorVolumeData format, i.e.
    in subgroups for each element, there are all tensor components stored
    for the same. The optional inputs `include` and `exclude` are lists of
    tensors to collate (or not).
    """
    # Use the first element to get a list of tensors and elements
    elements = list(hdf_group.keys())
    first_elem = elements[0]    
    all_tensors = list(hdf_group[first_elem].keys())    
    # Include only those tensors that the user specifies in include
    if type(include) is list:
        all_tensors = list(set(all_tensors).intersection(set(include)))
        if len(all_tensors) == 0:
            raise IOError("Did you want to include any tensors..?")
    elif include == 'all':
        pass
    else:
        raise IOError("Either provide a list of tensor components as `include` OR just 'all'")    
    # Exclude tensors the user requests for
    assert type(exclude) is list, "Provide an (empty or filled) list of tensors to exclude"
    all_tensors = list(set(all_tensors).difference(set(exclude)))
    # Collate and populate tensors
    for t in all_tensors:
        if verbose:
            print("... combining data for {}".format(t))
        if t not in combined_tensors:
            combined_tensors[t] = []
            for sd in elements:
                combined_tensors[t] = np.append(combined_tensors[t], hdf_group[sd][t][()])
        else:
            if verbose:
                print(" ... skipping data for {} as its already present".format(t))
    return combined_tensors

**Usage of above methods**

In [3]:
ct = {}
with h5py.File('ID_VolumeData0.h5', 'r') as f:
    base = 'element_data.vol'
    observation_id = list(f[base].keys())[0]
    h = f[base][observation_id]
    for t in list(h[list(h.keys())[0]].keys()):
        if 'Gauge' not in t:
            continue
        ct = combine_element_wise_spectre_data(h, include = [t],
                                               combined_tensors=ct)

... combining data for InitialGaugeH_t
... combining data for InitialGaugeH_x
... combining data for InitialGaugeH_y
... combining data for InitialGaugeH_z
... combining data for SpacetimeDerivInitialGaugeH_tt
... combining data for SpacetimeDerivInitialGaugeH_tx
... combining data for SpacetimeDerivInitialGaugeH_ty
... combining data for SpacetimeDerivInitialGaugeH_tz
... combining data for SpacetimeDerivInitialGaugeH_xt
... combining data for SpacetimeDerivInitialGaugeH_xx
... combining data for SpacetimeDerivInitialGaugeH_xy
... combining data for SpacetimeDerivInitialGaugeH_xz
... combining data for SpacetimeDerivInitialGaugeH_yt
... combining data for SpacetimeDerivInitialGaugeH_yx
... combining data for SpacetimeDerivInitialGaugeH_yy
... combining data for SpacetimeDerivInitialGaugeH_yz
... combining data for SpacetimeDerivInitialGaugeH_zt
... combining data for SpacetimeDerivInitialGaugeH_zx
... combining data for SpacetimeDerivInitialGaugeH_zy
... combining data for SpacetimeDe

In [6]:
ct.keys()

dict_keys(['InitialGaugeH_t', 'InitialGaugeH_x', 'InitialGaugeH_y', 'InitialGaugeH_z', 'SpacetimeDerivInitialGaugeH_tt', 'SpacetimeDerivInitialGaugeH_tx', 'SpacetimeDerivInitialGaugeH_ty', 'SpacetimeDerivInitialGaugeH_tz', 'SpacetimeDerivInitialGaugeH_xt', 'SpacetimeDerivInitialGaugeH_xx', 'SpacetimeDerivInitialGaugeH_xy', 'SpacetimeDerivInitialGaugeH_xz', 'SpacetimeDerivInitialGaugeH_yt', 'SpacetimeDerivInitialGaugeH_yx', 'SpacetimeDerivInitialGaugeH_yy', 'SpacetimeDerivInitialGaugeH_yz', 'SpacetimeDerivInitialGaugeH_zt', 'SpacetimeDerivInitialGaugeH_zx', 'SpacetimeDerivInitialGaugeH_zy', 'SpacetimeDerivInitialGaugeH_zz'])

In [8]:
with h5py.File('ID_VolumeData0.h5', 'r') as f:
    base = 'element_data.vol'
    observation_id = list(f[base].keys())[0]
    h = f[base][observation_id]
    for t in list(h[list(h.keys())[0]].keys()):
        if 'Spacetime' not in t:
            continue
        ct = combine_element_wise_spectre_data(h, include = [t],
                                               combined_tensors=ct)

... combining data for SpacetimeDerivInitialGaugeH_tt
 ... skipping data for SpacetimeDerivInitialGaugeH_tt as its already present
... combining data for SpacetimeDerivInitialGaugeH_tx
 ... skipping data for SpacetimeDerivInitialGaugeH_tx as its already present
... combining data for SpacetimeDerivInitialGaugeH_ty
 ... skipping data for SpacetimeDerivInitialGaugeH_ty as its already present
... combining data for SpacetimeDerivInitialGaugeH_tz
 ... skipping data for SpacetimeDerivInitialGaugeH_tz as its already present
... combining data for SpacetimeDerivInitialGaugeH_xt
 ... skipping data for SpacetimeDerivInitialGaugeH_xt as its already present
... combining data for SpacetimeDerivInitialGaugeH_xx
 ... skipping data for SpacetimeDerivInitialGaugeH_xx as its already present
... combining data for SpacetimeDerivInitialGaugeH_xy
 ... skipping data for SpacetimeDerivInitialGaugeH_xy as its already present
... combining data for SpacetimeDerivInitialGaugeH_xz
 ... skipping data for Spacet

In [9]:
ct.keys()

dict_keys(['InitialGaugeH_t', 'InitialGaugeH_x', 'InitialGaugeH_y', 'InitialGaugeH_z', 'SpacetimeDerivInitialGaugeH_tt', 'SpacetimeDerivInitialGaugeH_tx', 'SpacetimeDerivInitialGaugeH_ty', 'SpacetimeDerivInitialGaugeH_tz', 'SpacetimeDerivInitialGaugeH_xt', 'SpacetimeDerivInitialGaugeH_xx', 'SpacetimeDerivInitialGaugeH_xy', 'SpacetimeDerivInitialGaugeH_xz', 'SpacetimeDerivInitialGaugeH_yt', 'SpacetimeDerivInitialGaugeH_yx', 'SpacetimeDerivInitialGaugeH_yy', 'SpacetimeDerivInitialGaugeH_yz', 'SpacetimeDerivInitialGaugeH_zt', 'SpacetimeDerivInitialGaugeH_zx', 'SpacetimeDerivInitialGaugeH_zy', 'SpacetimeDerivInitialGaugeH_zz', 'SpacetimeMetric_tt', 'SpacetimeMetric_tx', 'SpacetimeMetric_ty', 'SpacetimeMetric_tz', 'SpacetimeMetric_xt', 'SpacetimeMetric_xx', 'SpacetimeMetric_xy', 'SpacetimeMetric_xz', 'SpacetimeMetric_yt', 'SpacetimeMetric_yx', 'SpacetimeMetric_yy', 'SpacetimeMetric_yz', 'SpacetimeMetric_zt', 'SpacetimeMetric_zx', 'SpacetimeMetric_zy', 'SpacetimeMetric_zz'])

In [ ]:
with h5py.File('ID_VolumeData0.h5', 'r') as f:
    base = 'element_data.vol'
    observation_id = list(f[base].keys())[0]
    h = f[base][observation_id]
    for t in list(h[list(h.keys())[0]].keys()):
        if 'Pi' not in t:
            continue
        ct = combine_element_wise_spectre_data(h, include = [t],
                                               combined_tensors=ct)

... combining data for Pi_tt
... combining data for Pi_tx
... combining data for Pi_ty
... combining data for Pi_tz
... combining data for Pi_xt
... combining data for Pi_xx
... combining data for Pi_xy
... combining data for Pi_xz
... combining data for Pi_yt
... combining data for Pi_yx
... combining data for Pi_yy


In [ ]:
ct.keys()

In [1]:
with h5py.File('ID_VolumeData0.h5', 'r') as f:
    base = 'element_data.vol'
    observation_id = list(f[base].keys())[0]
    h = f[base][observation_id]
    for t in list(h[list(h.keys())[0]].keys()):
        if 'Phi' not in t:
            continue
        ct = combine_element_wise_spectre_data(h, include = [t],
                                               combined_tensors=ct)

NameError: name 'h5py' is not defined

In [2]:
ct.keys()

NameError: name 'ct' is not defined

In [7]:
os.system("cp -rv EvolutionVolume0.h5 spectre_ID_VolumeData0.h5")

0

In [5]:
with h5py.File("spectre_ID_VolumeData0.h5", "a") as f:
    base = 'element_data.vol'
    observation_id = list(f[base].keys())[0]
    h = f[base][observation_id]
    add_tensor_component_to_spectre_group(h, ct, order_on=None)

In [14]:
os.system('cp -v {0} {1}'.format('spectre_ID_VolumeData0.h5',
                                 id_file.replace('ID_VolumeData0.h5', '')
                                 ))

0

**Development**

In [77]:
base = 'element_data.vol'

In [66]:
id_dir = '/home/prayush.kumar/wheeler_scratch/projects/'

In [67]:
id_file = id_dir + \
    'BoundaryConditionsGeneralizedHarmonic/gh_bctest_3d/perturbed_kerrschild/ID_VolumeData0.h5'

In [68]:
export_coords_file = id_dir + \
    'BoundaryConditionsGeneralizedHarmonic/gh_bctest_3d/perturbed_kerrschild/VolumeData0.h5'

In [69]:
os.system('cp -v {} .'.format(id_file))

0

In [70]:
os.system('cp -v {} .'.format(export_coords_file))

0

In [71]:
os.system("cp -rv VolumeData0.h5 spectre_ID_VolumeData0.h5")

0

In [102]:
f = h5py.File('spectre_ID_VolumeData0.h5', 'r')

In [73]:
observation_id = list(f[base].keys())[0]
for k in f[base][observation_id].keys():
    print(k, f[base][observation_id][k][()].shape)

DetInvJacobian(Logical,Inertial) (2239488,)
InertialCoordinates_x (2239488,)
InertialCoordinates_y (2239488,)
InertialCoordinates_z (2239488,)
connectivity (12582912,)
grid_names (70016,)
total_extents (9216,)


In [74]:
len(f[base][observation_id].keys())

7

File created by `ExportCoordinates3D` contains 7 datasets. These include the coordinates, `connectivity` and `total_extents`.

In [81]:
f[base][observation_id]['grid_names'][()][:100]

array([91, 66, 52, 55, 44, 40, 76, 50, 73, 51, 44, 76, 50, 73, 51, 44, 76,
       50, 73, 51, 41, 93, 58, 91, 66, 52, 55, 44, 40, 76, 50, 73, 51, 44,
       76, 50, 73, 51, 44, 76, 50, 73, 50, 41, 93, 58, 91, 66, 52, 55, 44,
       40, 76, 50, 73, 51, 44, 76, 50, 73, 51, 44, 76, 50, 73, 49, 41, 93,
       58, 91, 66, 52, 55, 44, 40, 76, 50, 73, 51, 44, 76, 50, 73, 50, 44,
       76, 50, 73, 50, 41, 93, 58, 91, 66, 52, 55, 44, 40, 76, 50],
      dtype=int8)

In [82]:
f[base][observation_id]['total_extents'][()][:100]

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9], dtype=uint64)

In [83]:
f[base][observation_id]['connectivity'][()][:100]

array([  0,   1,  10,   9,  81,  82,  91,  90,  81,  82,  91,  90, 162,
       163, 172, 171, 162, 163, 172, 171, 243, 244, 253, 252, 243, 244,
       253, 252, 324, 325, 334, 333, 324, 325, 334, 333, 405, 406, 415,
       414, 405, 406, 415, 414, 486, 487, 496, 495, 486, 487, 496, 495,
       567, 568, 577, 576, 567, 568, 577, 576, 648, 649, 658, 657,   9,
        10,  19,  18,  90,  91, 100,  99,  90,  91, 100,  99, 171, 172,
       181, 180, 171, 172, 181, 180, 252, 253, 262, 261, 252, 253, 262,
       261, 333, 334, 343, 342, 333, 334, 343, 342], dtype=int32)

In [111]:
f.close()

In [110]:
id_f = h5py.File('spectre_ID_VolumeData0.h5', 'r')
observation_id = list(id_f[base].keys())[0]
id_h = id_f[base][observation_id]

Now, we examine the ID file `ID_VolumeData0.h5` generated by Geoffrey

In [112]:
gl_f = h5py.File('ID_VolumeData0.h5', 'r')
observation_id = list(gl_f[base].keys())[0]
g = gl_f[base]
gl_h = gl_f[base][observation_id]

In [113]:
list(gl_h.keys())[0]

'[B0,(L2I0,L2I0,L2I0)]'

In [114]:
print(gl_h.attrs.keys())
print(gl_h.attrs['observation_value'])

<KeysViewHDF5 ['observation_value']>
0.0


In [115]:
len(gl_h.keys())

3072

There are clearly 3072 subdomains

In [154]:
print(list(gl_h[kk].keys()))

['InertialCoordinates_x', 'InertialCoordinates_y', 'InertialCoordinates_z', 'InitialGaugeH_t', 'InitialGaugeH_x', 'InitialGaugeH_y', 'InitialGaugeH_z', 'Phi_xtt', 'Phi_xtx', 'Phi_xty', 'Phi_xtz', 'Phi_xxt', 'Phi_xxx', 'Phi_xxy', 'Phi_xxz', 'Phi_xyt', 'Phi_xyx', 'Phi_xyy', 'Phi_xyz', 'Phi_xzt', 'Phi_xzx', 'Phi_xzy', 'Phi_xzz', 'Phi_ytt', 'Phi_ytx', 'Phi_yty', 'Phi_ytz', 'Phi_yxt', 'Phi_yxx', 'Phi_yxy', 'Phi_yxz', 'Phi_yyt', 'Phi_yyx', 'Phi_yyy', 'Phi_yyz', 'Phi_yzt', 'Phi_yzx', 'Phi_yzy', 'Phi_yzz', 'Phi_ztt', 'Phi_ztx', 'Phi_zty', 'Phi_ztz', 'Phi_zxt', 'Phi_zxx', 'Phi_zxy', 'Phi_zxz', 'Phi_zyt', 'Phi_zyx', 'Phi_zyy', 'Phi_zyz', 'Phi_zzt', 'Phi_zzx', 'Phi_zzy', 'Phi_zzz', 'Pi_tt', 'Pi_tx', 'Pi_ty', 'Pi_tz', 'Pi_xt', 'Pi_xx', 'Pi_xy', 'Pi_xz', 'Pi_yt', 'Pi_yx', 'Pi_yy', 'Pi_yz', 'Pi_zt', 'Pi_zx', 'Pi_zy', 'Pi_zz', 'SpacetimeDerivInitialGaugeH_tt', 'SpacetimeDerivInitialGaugeH_tx', 'SpacetimeDerivInitialGaugeH_ty', 'SpacetimeDerivInitialGaugeH_tz', 'SpacetimeDerivInitialGaugeH_xt', 'Space

In [ ]:
num_tensor_comps = [len(gl_h[k]) for k in list(gl_h.keys())]

In [ ]:
print(np.max(num_tensor_comps), np.min(num_tensor_comps))

There are 104 tensor components etc for each subdomain

In [117]:
kk = list(gl_h.keys())[0]
print(gl_h[kk]['connectivity'][()].shape)
gl_h[kk]['connectivity'][()][:100]

(4096,)


array([  0,   1,  10,   9,  81,  82,  91,  90,  81,  82,  91,  90, 162,
       163, 172, 171, 162, 163, 172, 171, 243, 244, 253, 252, 243, 244,
       253, 252, 324, 325, 334, 333, 324, 325, 334, 333, 405, 406, 415,
       414, 405, 406, 415, 414, 486, 487, 496, 495, 486, 487, 496, 495,
       567, 568, 577, 576, 567, 568, 577, 576, 648, 649, 658, 657,   9,
        10,  19,  18,  90,  91, 100,  99,  90,  91, 100,  99, 171, 172,
       181, 180, 171, 172, 181, 180, 252, 253, 262, 261, 252, 253, 262,
       261, 333, 334, 343, 342, 333, 334, 343, 342], dtype=int32)

In [118]:
id_h['connectivity'][()][:100]

array([  0,   1,  10,   9,  81,  82,  91,  90,  81,  82,  91,  90, 162,
       163, 172, 171, 162, 163, 172, 171, 243, 244, 253, 252, 243, 244,
       253, 252, 324, 325, 334, 333, 324, 325, 334, 333, 405, 406, 415,
       414, 405, 406, 415, 414, 486, 487, 496, 495, 486, 487, 496, 495,
       567, 568, 577, 576, 567, 568, 577, 576, 648, 649, 658, 657,   9,
        10,  19,  18,  90,  91, 100,  99,  90,  91, 100,  99, 171, 172,
       181, 180, 171, 172, 181, 180, 252, 253, 262, 261, 252, 253, 262,
       261, 333, 334, 343, 342, 333, 334, 343, 342], dtype=int32)

In [126]:
gl_n_elems = len(gl_h.keys())
gl_conn_per_elem = gl_h[kk]['connectivity'][()].shape[0]
id_n_conn = id_h['connectivity'][()].shape[0]

gl_n_elems * gl_conn_per_elem == id_n_conn

True

The product of `number of elements` and `no of connectivity entries for each element` should be equal to the `connectivity` entries in Geoffrey's data. It is, as it turns out.

In [134]:
all_tvals = []
for ii in range(gl_n_elems):
    kk = list(gl_h.keys())[ii]
    tval = np.all(gl_h[kk]['connectivity'][()][:] == id_h['connectivity'][()][ii*4096 : (ii+1)*4096])
    all_tvals.append(tval)
    print(ii, tval)
    if ii > 20: break

0 True
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
21 False


**The problem now is to match the connectivity entries from element-wise data to collated data**

In [148]:
test_ii = 110

# first, get gl data for this subdomain
kk = list(gl_h.keys())[test_ii]
print("using element: {}".format(kk))
gl_test_d = gl_h[kk]['connectivity'][()][:]

# then, get total data
id_test_d = id_h['connectivity'][()]

using element: [B1,(L2I2,L2I3,L2I2)]


In [149]:
id_test_d.shape[0] // gl_test_d.shape[0]

3072

In [150]:
all_tvals = []
for jj in range(gl_n_elems):
    tval = np.all(gl_test_d == id_test_d[jj*4096 : (jj+1)*4096])
    all_tvals.append(tval)

In [151]:
np.where(all_tvals)

(array([0]),)

**Apparently, all connectivity info is identical between elements in GL data!!!**

**Now, we work with the inertial coordinates. Those have to be mapped between data**

In [160]:
test_ii = 0

# first, get gl data for this subdomain
kk = list(gl_h.keys())[test_ii]
print("using element: {}".format(kk))
gl_test_d = gl_h[kk]['InertialCoordinates_x'][()][:]

# then, get total data
id_test_d = id_h['InertialCoordinates_x'][()]

using element: [B0,(L2I0,L2I0,L2I0)]


In [161]:
all_tvals = []
for jj in range(gl_n_elems):
    tval = np.all(gl_test_d == id_test_d[jj*4096 : (jj+1)*4096])
    all_tvals.append(tval)

/cvmfs/ligo-containers.opensciencegrid.org/lscsoft/conda/latest/envs/ligo-py36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  app.launch_new_instance()


In [162]:
np.where(all_tvals)

(array([], dtype=int64),)

In [ ]:
id_f.close()
gl_f.close()

In [ ]:
# Geoffrey's script to add soft links for the symmetric
# components of tensors in volume data file

import argparse
import h5py
import numpy as np
import shutil

if __name__ == "__main__":
    help = """Takes a spectre-volume-data-format HDF5 file
    that contains SpacetimeMetric, Pi, and Phi, and makes
    soft links for off-diagonal components. This script
    assumes that the data in the HDF5 file consits of
    the spacetime components tt, tx, ty, tz, xx, xy, xz, yy, yz, zz.
    The soft links map xt -> tx, yx -> xy, etc.
    """
    parser = argparse.ArgumentParser(description=help)
    parser.add_argument('--input', required=True,
                        help = """Name of HDF5 file""")
    args = parser.parse_args()

    spacetime_components = ["t", "x", "y", "z"]
    symmetric_tensors_in_file = ["Phi_x",  "Phi_y", "Phi_z", "Pi_",
                                 "SpacetimeMetric_"]

    file = h5py.File(args.input, 'a')
    base = "element_data.vol"
    for obs_id in file[base].keys():
        for tensor in symmetric_tensors_in_file:
            for i,comp1 in enumerate(spacetime_components):
                for j,comp2 in enumerate(spacetime_components):
                    if i > j:
                        file[base][obs_id][tensor+comp1+comp2] = \
                          h5py.SoftLink("/" + base + "/" + obs_id + "/"
                                        + tensor + comp2 + comp1)
    file.close()

In [ ]:
import argparse
import h5py
import numpy as np
import shutil

def rename_variables(legend_line):
    """Takes a string containing space-delimited tensor names and
    returns the restring with some tensors renamed:
        psi, kappa, H and GradH (i.e., flattened SpacetimeDerivH)
        ->
        SpacetimeMetric, Pi/Phi, GaugeH, and SpacetimeDerivGaugeH
    """
    legend_line = legend_line.replace('psi', 'SpacetimeMetric_')
    legend_line = legend_line.replace('kappat', 'Pi_')
    legend_line = legend_line.replace('kappax', 'Phi_x').replace('kappay', 'Phi_y').replace('kappaz', 'Phi_z')
    legend_line = legend_line.replace('GradH', 'SpacetimeDerivInitialGaugeH_')
    legend_line = legend_line.replace(' H', ' InitialGaugeH_')
    return legend_line.rstrip()

def get_spec_points(spectre_file):
    """Converts points from a file produced by spectre's ExportCoordinates
    to a flat list of points, suitable for SpEC to read in (e.g.,
    for interpolating data onto those points)"""
    observation_id = list(spectre_file['element_data.vol'].keys())[0]
    coords_dict = dict(spectre_file['element_data.vol'][observation_id])

    components = ['InertialCoordinates_x', 'InertialCoordinates_y', 'InertialCoordinates_z']
    dim = len(components)

    coords = []
    for component in components:
        coords.append([])

    for i,component in enumerate(components):
        coords[i].append(coords_dict[component])
    return np.transpose(np.array([np.concatenate(x) for x in coords]))

def write_spec_points_file(spectre_points_filename, spec_points_filename):
    """Read the coordinates from a spectre domain and write them in a
    file readable by spec's InterpolateToSpecifiedPoints. The input arguments
    are the name of a spectre points file (e.g. 'VolumeData0.h5') and
    the name of the spec points file to write (e.g. 'PointsList.txt')."""
    spectre_file = h5py.File(spectre_points_filename, 'r')
    points = get_spec_points(spectre_file)
    spectre_file.close()
    np.savetxt(spec_points_filename, points)

def insert_spec_data(spectre_points_filename, spec_data_filename):
    """Insert tensor data given in the spectre volume-data file
    named spec_data_filename (typically output by spec's
    InterpolateToSpecifiedPoints) into the spectre volume-data file named
    spectre_points_filename. Note that this function will rename some
    variables if it finds them by calling rename_variables()"""
    # Read the interpolated data into a numpy array
    data_to_insert = np.genfromtxt(spec_data_filename)

    # Get the legend
    spec_file = open(spec_data_filename, 'r')
    lines = spec_file.readlines()

    # spec output lists the components as a comment on the second line
    # in the format '# psitt psitx ...'
    legend_line = lines[1][2:]
    legend_line = rename_variables(legend_line)
    legend = legend_line.split(" ")

    legend_dict = {}
    for i, key in enumerate(legend):
        legend_dict[key] = i
    spec_file.close()

    # Open file read-only to determine observation_id
    spectre_file = h5py.File(spectre_points_filename, 'r')
    observation_id = list(spectre_file['element_data.vol'].keys())[0]
    spectre_file.close()

    # Open file ready to append data
    output_file = h5py.File(spectre_points_filename, 'a')

    # Loop over keys
    for key in legend_dict:
        print("Inserting " + key)
        spec_data = data_to_insert[:, legend_dict[key]]
        output_file['element_data.vol'][observation_id][key] = spec_data

    output_file.close()
    return legend_dict, data_to_insert

if __name__ == "__main__":
    help = """Convert data between spec and spectre. This script reads
    the spectre domain from a spectre volume-data file specified by
    --spectre-points-filename, output e.g. by
    spectre's ExportCoordinates3D executable. If the option
    --output-spec-points-filename is given, this script will write the
    spectre grid coordinates into a file to be read by spec's
    InterpolateToSpecifiedPoints. If the option
    --spec-data-to-insert-filename is given, this script will read data
    from the specified file, which contains output from spec's
    InterpolateToSpecifiedPoints (with option DumpAllDataIntoSingleFile=yes).
    Then, this script will insert that data into a copy of the spectre
    volume-data file specified by --output-spectre-points-filename.
    """
    parser = argparse.ArgumentParser(description=help)
    parser.add_argument('--spectre-points-filename', required=True,
                        help = """Name of spectre volume data file
                        containing the x,y,z coordinates of a
                        spectre domain.""")
    parser.add_argument('--output-spec-points-filename', required=False,
                        help = """If specified, output coordinates in spec
                        format to this file""")
    parser.add_argument('--spec-data-to-insert-filename', required=False,
                        help = """If specified, insert data from this file
                        into the spectre volume data file given by
                        --output-spectre-points-filename (if specified).""")
    parser.add_argument('--output-spectre-points-filename', required=False,
                        help = """If --spec-data-to-insert-filename is
                        specified and this option is specified,
                        copy the file given by --spectre-points-filename
                        to a new file with name
                        --output-spectre-points-filename, and then insert
                        the data contained in the file given by
                        --spec-data-to-insert-filename.""")
    args = parser.parse_args()

    if args.output_spec_points_filename is not None:
        write_spec_points_file(args.spectre_points_filename,
                               args.output_spec_points_filename)

    if args.spec_data_to_insert_filename is not None:
        print("Inserting data from " + args.spec_data_to_insert_filename)
        if args.output_spectre_points_filename is not None:
            print("Inserting into " + args.output_spectre_points_filename)
            shutil.copyfile(args.spectre_points_filename,
                            args.output_spectre_points_filename)
            insert_spec_data(args.output_spectre_points_filename,
                             args.spec_data_to_insert_filename)